In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

# pd.set_option('display.max_columns', None)
Allows you to see all columns :)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data_og = pd.read_csv('../data/all_evals.csv')

In [ ]:
data = pd.read_csv('../data/all_evals.csv')
#data.head()

Remove code columns

In [ ]:
code_cols = []
pattern = re.compile('Code')

for col in data.columns:
    if re.search(pattern,col):
        code_cols.append(col)
#end

data = data.drop(columns = code_cols)
#data.head()

Get rid of duplicate rows

In [ ]:
data = data.loc[~data.duplicated()]

In [ ]:
data = data.reset_index(drop = True)

Clean up RDI columns

In [ ]:
### This is for data imported as a CSV. Will not work with an imported Excel file

RDI = re.compile('RDI') # r'RDI'
pattern = r'\d{1,2}\/90'
#new_vals = []
#new_cols = {}

for col in data.columns:
    if re.search(RDI,col):
        print('NEW COLUMN:',col)
        ind = 0
        
        for value in data[col]:
            print('Compare',value,'to',data[col][ind])
            
            if re.match(pattern,str(value)):
                new_val = str(value)[:-3]
                data.loc[ind,col] = int(new_val)
                print(ind, 'Match', new_val)
                
            elif (str(value)[0] == '<') or (str(value)[0] == '>'):
                new_val = str(value)[1:-3]
                data.loc[ind,col] = int(new_val)
                print(ind, '<>', new_val)
                
            elif type(value) == float:
                ind += 1
                print(ind, 'NaN', new_val)
                continue
            
            else:
                date = datetime.datetime.strptime(str(value), '%b-%y')
                new_val = str(date.month)
                data.loc[ind,col] = int(new_val)
                print(ind, 'date', new_val)
            #end
            
            ind += 1

#end

In [ ]:
#data.head(20)

Clean up other columns

In [ ]:
# Make sure that pandas, numpy, and re are imported!

def clean_col(data, word): # Do not use for RDI. Column name can work for 'word' (make sure word is a string!) 
    
    pattern = re.compile(word)

    for col in data.columns:
        if re.search(pattern,col): # searches for word within column names
            ind = 0

            for value in data[col]:
                
                try:
                    if (str(value)[0] == '<') or (str(value)[0] == '>'):    # if value starts with < or > removes symbol
                        new_val = str(value)[1:]
                        data.loc[ind,col] = float(new_val)
                
                    elif (value == '') or (value == '--') or (value == '–'):    # if blank, returns null
                        new_val = np.nan
                        data.loc[ind,col] = new_val

                    else:    # if value can be cast as float, it will (including nulls)
                        new_str = str(value).replace('– ', '-').replace('--','-').replace(' ','-') # makes sure that any negatives will be read correctly
                        new_val = float(new_str)
                        data.loc[ind,col] = new_val
                    
                    ind += 1
                        
                except:
                    if type(value) == str:    # if value is a string that doesn't start with <,> -> keeps string as is
                        ind += 1
                        continue
                    
                    else:    # if all else fails, puts in null
                        data.loc[ind,col] = np.nan
                    
                    ind += 1

#end

In [ ]:
# 'Percentile', 'PR', 'AE', 'NCE', 'Z-Score', 'T-Score', 'RS', 'SS', 'CSS'
#clean_col(data,'Percentile')

In [ ]:
clean_col(data,'Adaptive')

In [ ]:
clean_col(data,'Social')

In [ ]:
clean_col(data,'Communication')

In [ ]:
clean_col(data,'Motor')

In [ ]:
clean_col(data,'Cognitive')

In [ ]:
clean_col(data,'BDI-3')

In [ ]:
data.head(50)

In [ ]:
data.loc[data['Adaptive Z-Score'] < 0]

In [ ]:
data['Adaptive Percentile Rank'].astype(float)

In [ ]:
data_desc = data.describe()
data_desc.columns

In [ ]:
data.to_csv('../data/all_evals_clean.csv')

In [ ]:
list(data.columns)

In [ ]:
cols_to_convert = []

pd.to_numeric

In [ ]:
print(data_og['Adaptive Z-Score'])

In [ ]:
list(data_og['Adaptive Z-Score'])

In [ ]:
float(np.nan)

In [ ]:
new_str = str(np.nan).replace('  ',' ')
print(new_str, type(new_str))
float(new_str)

In [ ]:
data.head(30)

In [ ]:
data.loc[0,'Adaptive Percentile Rank']

### Data is now clean!

Next, let's create sub-dataframes based on what domain the column is in.

In [ ]:
list(data.columns)

In [ ]:
def slice_data(data, domain):
    
    pattern = re.compile(domain)
    
    new_cols = []
    
    for col in data.columns:
        if re.search(pattern,col):
            new_cols.append(col)
    #end
    
    new_df = data[new_cols]
    
    return new_df

In [ ]:
domains = ['Adaptive','Social','Communication','Motor','Cognitive','Total']
df_dict = {}

for domain in domains:
    new_df = slice_data(data,domain)
    df_dict[domain] = new_df
#end

print(df_dict.keys())

In [ ]:
adapt = df_dict['Adaptive']
soc   = df_dict['Social']
comm  = df_dict['Communication']
motor = df_dict['Motor']
cogn  = df_dict['Cognitive']
total = df_dict['Total']

We have now sliced the data based on domain! Now let's tackle the questions.

## 1. In which domains (and sub-domains) are children performing highest and lowest?

In [ ]:
adapt

In [ ]:
adapt_0 = adapt.fillna(0)
adapt_0

In [ ]:
list(adapt.columns)

In [ ]:
adapt_0.describe()

In [ ]:
adapt.describe()

Replacing nulls with 0's impacts the last few columns the most, skewing the average. As such, I won't be replacing the null values for the rest of the dataframes.

Also, several columns are missing from the describe table. This is because the rows datatypes are objects, not numeric. So I have to fix those columns before we move on.

In [ ]:
adapt2 = adapt.drop(columns = ['Adaptive 95% Confidence Interval', 'Adaptive-Self Care CSS 90%', 'Adaptive-Personal Responsibility CSS 90%'])
adapt2

In [ ]:
ind=0
for value in adapt2['Adaptive RDI']:
    print(ind,value,'\t',str(value)[:-3])
    adapt2.loc[ind,'Adaptive RDI'] = str(value)[:-3]
    ind += 1

In [ ]:
adapt2

In [ ]:
adapt2 = adapt2.replace('<0.1','0.1').replace('<1','1').replace('>99','99').replace('',0)
adapt2

In [ ]:
adapt2.astype(float)

In [ ]:
'123456789'[:-3]

This has become tedious to drop the columns I don't want. I'm going to define a nenw function that will only give me columns I want.

In [ ]:
def slice_numeric_data(data, domain):
    date    = re.compile('Date')
    percent = re.compile('%')
    examine = re.compile('Examiner')
    
    pattern = re.compile(domain)
    
    new_cols = []
    
    for col in data.columns:
        if (re.search(pattern,col)):
            if (not re.search(date,col)) & (not re.search(percent,col)) & (not re.search(examine,col)):
                new_cols.append(col)
    #end
    
    new_df = data[new_cols]
    
    return new_df

In [ ]:
adapt3 = slice_numeric_data(data, 'Adaptive')
adapt3.head(20)

In [ ]:
adapt3.astype(float)

In [ ]:
df_dict = {}

for domain in domains:
    new_df = slice_numeric_data(data,domain)
    df_dict[domain] = new_df
#end

print(df_dict.keys())

In [ ]:
adapt = df_dict['Adaptive']
soc   = df_dict['Social']
comm  = df_dict['Communication']
motor = df_dict['Motor']
cogn  = df_dict['Cognitive']
total = df_dict['Total']

In [ ]:
adapt

In [ ]:
adapt.replace('<1','1').replace('>99','99').replace('<24','24')